# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, choice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'Hyperdriveversion'

experiment=Experiment(ws, experiment_name)

In [ ]:
amlcompute_cluster_name = "hyperdrive-compute"

# Verify that cluster does not exist already
try:
    compute_instance = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_instance = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_instance.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [ ]:
found = False
key = "**HeartFailure Dataset**"
description_text = "**Heart Failure DataSet for Kaggle**"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/maulingogri/Azure-Udacity-MLE-ND-Capstone/master/data/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
# Split data into train and score sets
train, score = dataset.random_split(percentage=0.75, seed=121)

In [ ]:
train, score = dataset.random_split(percentage=0.75, seed=121)
  
data = train.to_pandas_dataframe()
y = data['DEATH_EVENT']
x = data.drop('DEATH_EVENT', 1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=121)

In [ ]:
y_train.head(5)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.01)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": loguniform(-4, 3),
    "--max_iter": choice(100, 250, 400, 550, 700, 850, 100)
})


#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory=".",
              entry_script='train.py',
              compute_target = compute_instance)

hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=6)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_model = hyperdrive_run.get_best_run_by_primary_metric()
best_model_metrics = best_model.get_metrics()
parameter_values = best_model.get_details() ['runDefinition']['arguments']

In [ ]:
print('Best Model Id: ', best_model.id)
print(' -Accuracy:', best_model_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

In [ ]:
#TODO: Save the best model
from azureml.core.model import Model
best_model.register_model(model_path='outputs/hd_model.joblib', model_name='heart_hd_model', model_framework=Model.Framework.SCIKITLEARN)
model = ws.models['heart_hd_model']
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
folder_name = 'heart_failure_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_heart_failure.py")

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('heart_hd_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # Get a prediction from the model
    predictions = model.predict(np_data)
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['no-failure', 'failure']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = folder_name + "/heartfailure_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

In [ ]:
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)
service_name = 'hd-service'
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)
service = Model.deploy(ws, name=service_name, models=[model], inference_config= inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
# Enable AppInsights
service.update(enable_app_insights=True)
print('AppInsights enabled!')
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
endpoint = service.scoring_uri
primary, secondary = service.get_keys()
print(primary)
print(endpoint)

In [ ]:
import requests
import json

scoring_uri = service.scoring_uri
# Create new data for inferencing
x_new = [[55,0,7861,0,38,0,263358.03,1.1,136,1,0,6],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }
headers['Authorization'] = f'Bearer {primary}'

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
print('Service deleted.')